In [7]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [8]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/data/WeatherPy_Challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Havelock,US,2020-05-09 23:55:20,34.88,-76.90,61.00,41,0,1.01,clear sky,0,0
1,1,Saskylakh,RU,2020-05-09 23:55:20,71.92,114.08,23.27,97,100,9.40,light snow,0,0
2,2,Atuona,PF,2020-05-09 23:55:21,-9.80,-139.03,80.78,78,51,18.97,light rain,0,0
3,3,Grindavik,IS,2020-05-09 23:55:21,63.84,-22.43,37.40,55,0,4.70,clear sky,0,0
4,4,Sydney,AU,2020-05-09 23:52:01,-33.87,151.21,60.01,41,20,21.92,few clouds,0,0


In [9]:
#remove extra City_ID index column
city_data_df = city_data_df.drop(columns=["City_ID"])
city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,Havelock,US,2020-05-09 23:55:20,34.88,-76.90,61.00,41,0,1.01,clear sky,0,0
1,Saskylakh,RU,2020-05-09 23:55:20,71.92,114.08,23.27,97,100,9.40,light snow,0,0
2,Atuona,PF,2020-05-09 23:55:21,-9.80,-139.03,80.78,78,51,18.97,light rain,0,0
3,Grindavik,IS,2020-05-09 23:55:21,63.84,-22.43,37.40,55,0,4.70,clear sky,0,0
4,Sydney,AU,2020-05-09 23:52:01,-33.87,151.21,60.01,41,20,21.92,few clouds,0,0


In [10]:
city_data_df.dtypes

City                         object
Country                      object
Date                         object
Lat                         float64
Lng                         float64
Max Temp                    float64
Humidity                      int64
Cloudiness                    int64
Wind Speed                  float64
Current Description          object
Rain inches (last 3 hrs)      int64
Snow inches (last 3 hrs)      int64
dtype: object

In [11]:
# Ask the customer to add a minimum and maximum temperature value.

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

# ask the customer to answer if they want it to be raining or not; loop and return error for 
# answers other than 'yes' or 'no'

raining_pref = None
snowing_pref = None

raining = input("Do you want it to be raining? (yes/no) ")
while raining_pref == None:
    if raining == "yes":
        raining_pref = True
        continue
    elif raining == "no":
        raining_pref = False
        continue
    else:
        print("Error")
        raining = input("Please enter 'yes' or 'no' ")                      
                        
snowing = input("Do you want it to be snowing? (yes/no) ")
while snowing_pref == None:
    if snowing == "yes":
        snowing_pref = True
        continue
    elif snowing == "no":
        snowing_pref = False
        continue
    else:
        print("Error")
        snowing = input("Please enter 'yes' or 'no' ")
        

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 78
Do you want it to be raining? (yes/no) No
Error
Please enter 'yes' or 'no' no
Do you want it to be snowing? (yes/no) no


In [12]:
# create new dataframe based on min and max temperature preference
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)].dropna()

# refine new dataframe based on rain preference
if raining_pref == True:
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Rain inches (last 3 hrs)"] > 0)]
else:
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Rain inches (last 3 hrs)"] == 0)]

# refine new dataframe based on snow preferece
if snowing_pref == True:
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Snow inches (last 3 hrs)"] > 0)]
else:
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Snow inches (last 3 hrs)"] == 0)]
    
preferred_cities_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
12,Jalu,LY,2020-05-09 23:55:22,29.03,21.55,67.42,24,26,7.14,scattered clouds,0,0
17,Pitimbu,BR,2020-05-09 23:55:22,-7.47,-34.81,75.20,88,20,3.36,few clouds,0,0
27,Amapa,BR,2020-05-09 23:55:23,1.00,-52.00,74.08,97,84,2.53,light rain,0,0
29,Rikitea,PF,2020-05-09 23:55:23,-23.12,-134.97,75.85,69,65,4.03,broken clouds,0,0
32,Terra Santa,BR,2020-05-09 23:55:23,-2.10,-56.49,77.68,85,67,1.23,broken clouds,0,0
36,Arraial Do Cabo,BR,2020-05-09 23:55:24,-22.97,-42.03,71.73,64,0,12.21,clear sky,0,0
42,Touros,BR,2020-05-09 23:55:24,-5.20,-35.46,77.00,83,20,4.70,few clouds,0,0
44,Sao Filipe,CV,2020-05-09 23:55:25,14.90,-24.50,73.74,74,98,8.72,overcast clouds,0,0
46,Nara,JP,2020-05-09 23:54:25,34.69,135.80,69.01,72,75,1.12,broken clouds,0,0
51,Saint-Pierre,RE,2020-05-09 23:55:25,-21.34,55.48,69.80,73,0,8.05,clear sky,0,0


In [13]:
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng", "Current Description"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
12,Jalu,LY,67.42,29.03,21.55,scattered clouds,
17,Pitimbu,BR,75.20,-7.47,-34.81,few clouds,
27,Amapa,BR,74.08,1.00,-52.00,light rain,
29,Rikitea,PF,75.85,-23.12,-134.97,broken clouds,
32,Terra Santa,BR,77.68,-2.10,-56.49,broken clouds,
36,Arraial Do Cabo,BR,71.73,-22.97,-42.03,clear sky,
42,Touros,BR,77.00,-5.20,-35.46,few clouds,
44,Sao Filipe,CV,73.74,14.90,-24.50,overcast clouds,
46,Nara,JP,69.01,34.69,135.80,broken clouds,
51,Saint-Pierre,RE,69.80,-21.34,55.48,clear sky,


In [14]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [15]:
# Iterate through the cities in preference list.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
12,Jalu,LY,67.42,29.03,21.55,scattered clouds,Jalu Hotel
17,Pitimbu,BR,75.20,-7.47,-34.81,few clouds,Reserva do Abiaí
27,Amapa,BR,74.08,1.00,-52.00,light rain,
29,Rikitea,PF,75.85,-23.12,-134.97,broken clouds,Pension Maro'i
32,Terra Santa,BR,77.68,-2.10,-56.49,broken clouds,Igreja da paz
36,Arraial Do Cabo,BR,71.73,-22.97,-42.03,clear sky,Pousada Porto Praia
42,Touros,BR,77.00,-5.20,-35.46,few clouds,INN NEW HORIZON
44,Sao Filipe,CV,73.74,14.90,-24.50,overcast clouds,Tortuga B&B
46,Nara,JP,69.01,34.69,135.80,broken clouds,Guesthouse Nara Komachi
51,Saint-Pierre,RE,69.80,-21.34,55.48,clear sky,Lindsey Hôtel


In [12]:
# configure gmaps to use Google API key
gmaps.configure(api_key=g_key)

In [14]:
# Plot of map with hotels in cities of customer prefernce
# get latitude and longitute into an array for marker
locations = preferred_cities_df[["Lat", "Lng"]]
# assign figure variable
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# add marker layer with locations
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

In [16]:
# get data for infobox
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [17]:
# Plot of map with hotels in cities of customer prefernce and add information box
# get latitude and longitute into an array for marker
locations = preferred_cities_df[["Lat", "Lng"]]
# assign figure variable
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# add marker layer with locations
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
# create the output file (CSV)
output_data_file = "weather_data/data/WeatherPy_vacation.csv"
# export the City_Data into a CSV
hotel_df.to_csv(output_data_file, index_label="City_ID")